## 示例：用特定于分组的值填充缺失值

对于缺失数据的清理工作，有时你会用dropna将其替换掉，而有时则可能会希望用一个固定值或由数据集本身所衍生出来的值去填充NA值。这时就得使用fillna这个工具了。在下面这个例子中，我用平均值去填充NA值：

In [2]:
import pandas as pd
import numpy as np

In [4]:
s = pd.Series(np.random.randn(6))


s[::2] = np.nan

s

0         NaN
1   -0.634215
2         NaN
3   -0.390592
4         NaN
5    0.176936
dtype: float64

In [7]:
s.fillna(s.mean())

0   -0.282624
1   -0.634215
2   -0.282624
3   -0.390592
4   -0.282624
5    0.176936
dtype: float64

假设你需要对不同的分组填充不同的值。一种方法是将数据分组，并使用apply和一个能够对各数据块调用fillna的函数即可。下面是一些有关美国几个州的示例数据，这些州又被分为东部和西部：

In [10]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4

In [12]:
data = pd.Series(np.random.randn(8), index=states)

data

Ohio          1.939193
New York     -0.998461
Vermont      -0.488331
Florida      -1.340052
Oregon        0.652437
Nevada       -1.368933
California   -0.187816
Idaho         1.983497
dtype: float64

['East'] * 4产生了一个列表，包括了['East']中元素的四个拷贝。将这些列表串联起来。

将一些值设为缺失：

In [13]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

data

Ohio          1.939193
New York     -0.998461
Vermont            NaN
Florida      -1.340052
Oregon        0.652437
Nevada             NaN
California   -0.187816
Idaho              NaN
dtype: float64

In [14]:
data.groupby(group_key).mean()

East   -0.133107
West    0.232310
dtype: float64

我们可以用分组平均值去填充NA值:

In [17]:
fill_mean = lambda g: g.fillna(g.mean())


data.groupby(group_key).apply(fill_mean)

/tmp/ipykernel_89403/3003259577.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_mean)


Ohio          1.939193
New York     -0.998461
Vermont      -0.133107
Florida      -1.340052
Oregon        0.652437
Nevada        0.232310
California   -0.187816
Idaho         0.232310
dtype: float64

另外，也可以在代码中预定义各组的填充值。由于分组具有一个name属性，所以我们可以拿来用一下：

In [20]:
fill_values = {'East': 0.5, 'West': -1}

fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

/tmp/ipykernel_89403/2932218286.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_func)


Ohio          1.939193
New York     -0.998461
Vermont       0.500000
Florida      -1.340052
Oregon        0.652437
Nevada       -1.000000
California   -0.187816
Idaho        -1.000000
dtype: float64